# Project 5: Working with Pandas and SQL Databases (Movies Dataset)

## Creating an SQLite Database

In [ ]:
import sqlite3 as sq3

**Creation of the connection "Con"**

In [ ]:
con = sq3.connect("movies.db")  # creation of the database "movies.db"

In [ ]:
con

**Execution of the first sql query: "Select * FROM sqlite_master" (* = ALL). We will get an empty list because we didn't create any table yet**

In [ ]:
con.execute("Select * FROM sqlite_master").fetchall()  

**One more sql query:**

In [ ]:
con.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()

**Let's close the connection to the database (after this we cannot run queries anymore):**

In [ ]:
con.close()

## Loading Data from DataFrames into an SQLite Database

In [ ]:
import pandas as pd
import json
import sqlite3 as sq3

In [ ]:
with open("some_movies.json") as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
pd.set_option("display.max_rows", 3, "max_columns", 31)
#pd.set_option('display.max_rows', 8)

df = pd.json_normalize(data, sep = "_")

In [ ]:
df

**Let's create a dataframe which contains only the most important columns (id, title, revenue, etc). Whenever you create a subset of a pandas DataFrame and then modify the subset, the original DataFrame will also be modified. For this reason, it's always a good idea to use .copy() when subsetting so that any modifications you make to the subset won't also be made to the original DataFrame**

In [ ]:
movies = df[["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]].copy()
movies

In [ ]:
movies.info()

**Let's change the datatype of the date into "datetime"**

In [ ]:
movies.release_date = pd.to_datetime(df.release_date)

In [ ]:
movies.info()

**It is more convenient to have "revenue" and "budget" in terms of milion US dollars:**

In [ ]:
movies.revenue = df.revenue/1000000
movies.budget = df.budget/1000000

In [ ]:
movies

**Let's create a second dataframe: this dataframe contains only information about the votes. IMPORTANT: this dataframe has to contain the "id" column too, because that's how this dataframe and the "movies" dataframe can be linked to each other!**

In [ ]:
votes = df[["id", "vote_count", "vote_average"]].copy()
votes

**The "genres" column is a nestfied column (NOT STRINGFIED: only nested):**

In [ ]:
data

**Let's create a dataframe "genre" which contains only for information about "Genre". In order to do this, we will use the json file "data"**:

In [ ]:
genres = pd.json_normalize(data = data, record_path = "genres", meta = "id", record_prefix = "genre_")
genres

**Same process to create the new dataframe "prod"(production company):**

In [ ]:
prod = pd.json_normalize(data = data, record_path = "production_companies", meta = "id", record_prefix = "comp_")
prod

In [ ]:
con = sq3.connect("movies.db")

In [ ]:
con

In [ ]:
movies.to_sql("Movies", con, index = False)

In [ ]:
votes.to_sql("Votes", con, index = False)

In [ ]:
genres.to_sql("Genres", con, index = False)

In [ ]:
prod.to_sql("Prod", con, index = False)

In [ ]:
con.execute("Select * FROM sqlite_master").fetchall()

In [ ]:
con.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()

In [ ]:
con.close()

## Loading Data from SQLite Databases into DataFrames

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con = sq3.connect("movies.db")

In [ ]:
con.execute("Select * FROM sqlite_master").fetchall()

In [ ]:
pd.read_sql("SELECT * FROM Movies", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies", con).info()

In [ ]:
df = pd.read_sql("SELECT * FROM Movies", con, index_col = "id", parse_dates = "release_date")
df.info()

In [ ]:
genres = pd.read_sql("SELECT * FROM Genres", con, index_col = "id")
genres

In [ ]:
con.close()

##  Some Simple SQL Queries

In [ ]:
import sqlite3 as sq3

In [ ]:
con = sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM Movies", con)

In [ ]:
pd.read_sql("SELECT * \
            FROM Movies", con)

**Sql query to select specific columns:**

In [ ]:
pd.read_sql("SELECT id, revenue, release_date FROM Movies", con)

**Query for summary statistics:**

In [ ]:
pd.read_sql("SELECT sum(revenue) FROM Movies", con)

In [ ]:
con.execute("SELECT sum(revenue) FROM Movies").fetchall()[0][0]

**Check the non missing values of a specific table (Dataframe:column; Sql:table)**:

In [ ]:
pd.read_sql("SELECT count(title) FROM Movies", con)

In [ ]:
pd.read_sql("SELECT count(belongs_to_collection_name) FROM Movies", con)

In [ ]:
pd.read_sql("SELECT count(*) FROM Movies", con)

**Calculate the mean of the budget of all 18 movies:**

In [ ]:
pd.read_sql("SELECT avg(budget) FROM Movies", con)

In [ ]:
con.close()

## Some more SQL Queries

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con = sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE id = 597", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE revenue > 2000", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE revenue > 1500 AND budget < 200", con)

In [ ]:
pd.read_sql("SELECT MIN(budget) FROM Movies WHERE revenue > 1250", con)

**SELECT DINSTINCT**
**This query gives the single values, which means that if there are duplicates, only one value will be shown:**

In [ ]:
pd.set_option("display.max_rows", 6)
pd.read_sql("SELECT DISTINCT title FROM Movies", con)

In [ ]:
pd.read_sql("SELECT DISTINCT belongs_to_collection_name FROM Movies", con)

**ORDER BY: ordering values in ascending or descending way**

In [ ]:
pd.read_sql("SELECT * FROM Movies ORDER BY budget DESC", con)

**"WHERE...IS NULL" condition: here below we will obtain the movies that are not part of a collection/saga**

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name IS NULL", con)

**And here below the other way 'round: movies which are part of a collection**

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name IS NOT NULL", con)

**"WHERE...LIKE": obtaining rows that contain a specific word**

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE title LIKE 'Avengers%'", con)

In [ ]:
con.close()

## Join Queries

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con = sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM Movies", con)

In [ ]:
pd.read_sql("SELECT * FROM Votes", con)

**We want to join the table "Movies" and "Votes", which are connected by the column "id". We have in total four commands (SELECT, FROM, JOIN, ON). As best practice and rule of thumb they will be split in four lines. This makes the code neat and easy to read:**

In [ ]:
pd.read_sql("SELECT * \
            FROM Movies \
            JOIN Votes \
            ON Movies.id=Votes.id", con)

In [ ]:
pd.read_sql("SELECT * \
            FROM Votes \
            JOIN Movies \
            ON Movies.id=Votes.id", con)

In [ ]:
pd.read_sql("SELECT Movies.id, Movies.title, Votes.vote_average \
            FROM Movies \
            JOIN Votes \
            ON Movies.id=Votes.id", con, index_col = "id")

In [ ]:
pd.read_sql("SELECT Movies.id, Movies.title, Votes.vote_average \
            FROM Movies \
            JOIN Votes \
            ON Movies.id=Votes.id \
            WHERE Votes.vote_average > 8", con, index_col = "id")

In [ ]:
pd.read_sql("SELECT Movies.id, Movies.title, Movies.budget, Votes.vote_average \
            FROM Movies \
            JOIN Votes \
            ON Movies.id=Votes.id \
            WHERE Votes.vote_average > 8 \
            ORDER BY Movies.budget ASC", con, index_col = "id")

In [ ]:
con.close()

## Final Case Study

**What are the most successful production companies in terms of revenue?**

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 8)
import sqlite3 as sq3

In [ ]:
con = sq3.connect("movies.db")

**Let's take a look at the "prod" table (it's the table of the production companies). "comp_id" is the id of the company, while the last column "id" is the id of the movie (let's never forget that all these tables are connected by the id):**

In [ ]:
pd.read_sql("SELECT *\
            FROM Prod", con)

**Let's take a look at the "movies" table:**

In [ ]:
df = pd.read_sql("SELECT * \
            FROM Movies", con)
df

**Take a look at the sql query below:it doesn't use the primary key "id" to connect the two tables "Prod" and "Movies". Result: we will obtain all possible combinations of values (52x18)!
This kind of Dataframe is completely wrong of course, it doesn't deliver correct information.**

In [ ]:
df = pd.read_sql("SELECT * \
            FROM Prod \
            LEFT JOIN Movies", con)
df

**Sometimes a movie is the result of the collaboration of multiple production companies. In this project we have 52 production companies and 18 movies. We want to investigate which movie was produced by which company. We can obtain that with the LEFT JOIN query: we obtain all the records of the left table ("prod" in this case) combined to the corresponding records of the right table ("movies"). This is possible only thanks to the "id" column, which is in both tables. Only "id" we couldn't determine which company is related to which movie, because "id", which is in both table, is the unique key that is assigned to the single movie. Just "JOIN" (which is by default an "INNER JOIN") would work too, but when there are complex projects it's better to use LEFT JOIN **  

In [ ]:
pd.set_option('display.max_rows', 60)

df = pd.read_sql("SELECT Prod.id, Prod.comp_name, Movies.revenue, Movies.title \
            FROM Prod \
            LEFT JOIN Movies \
            ON Prod.id=Movies.id", con)
df.head(10)

In [ ]:
#df.to_dict()

In [ ]:
df.groupby(['id', 'title', 'revenue'])['comp_name'].apply(list)
df.head(10)

In [ ]:
pd.set_option('display.max_rows', 20)

df = pd.read_sql("SELECT Prod.id, Prod.comp_name, Movies.revenue, Movies.title \
            FROM Prod \
            LEFT JOIN Movies \
            ON Prod.id=Movies.id\
            GROUP BY Prod.id, Movies.revenue, Movies.title\
            ORDER BY Movies.revenue DESC", con)
df

In [ ]:
df.info()

**The most successful production companies**

In [ ]:
df.groupby("comp_name").revenue.sum().sort_values(ascending = False)

**Production companies only for the movie "Titanic"**

In [ ]:
pd.read_sql("SELECT  Prod.comp_name \
            FROM Prod \
            LEFT JOIN Movies \
            ON Prod.id=Movies.id \
            WHERE Movies.title = 'Titanic'", con)

**What genre is each movie? Let's join the table "Genres" with the table "Movie":**

In [ ]:
df2 = pd.read_sql("SELECT Genres.id, Genres.genre_name, Movies.revenue, Movies.title \
            FROM Genres \
            LEFT JOIN Movies \
            ON Genres.id=Movies.id", con)
df2

**Which genres makes more money? Let's group by genre:**

In [ ]:
df2.groupby("genre_name").revenue.sum().sort_values(ascending = False)

**Let's obtain the genre only for the movie "Frozen II":**

In [ ]:
pd.read_sql("SELECT Genres.genre_name \
            FROM Genres \
            LEFT JOIN Movies \
            ON Genres.id=Movies.id \
            WHERE Movies.title = 'Frozen II'", con)

In [ ]:
con.close()